In [1]:
import mlflow
import mlflow.sklearn


import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score, roc_curve


import warnings


warnings.filterwarnings('ignore')

load_dir = '../data/gold/'

# mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local

In [2]:
#FIXME - roc curve and confusion matrix are stored in the wrong location


In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment('fraud_logreg_base_model_reduced')

<Experiment: artifact_location='/home/maldu/dscience/projects/fraud_detection/experiments/artifacts_local/1', creation_time=1723977812939, experiment_id='1', last_update_time=1723977812939, lifecycle_stage='active', name='fraud_logreg_base_model_reduced', tags={}>

In [4]:
data = pd.read_parquet('../data/gold/df_fraud_final.parquet', engine= 'fastparquet')
data

,step,type,amount,nameOrig,oldbalanceOrig,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,day_of_month,amount_range,diffbalanceOrig,diffbalanceDest
index,,,,,,,,,,,,,,
1044,1,PAYMENT,4.518420e+03,C1504321715,8.790000e+03,4.271580e+03,M910123336,0.000000e+00,0.000000e+00,no_fraud,1,1.000-10.000,-4518.419922,0.00000
439,1,CASH_IN,1.314093e+05,C1476235721,8.491742e+06,8.623151e+06,C1068824137,2.902760e+05,2.650924e+05,no_fraud,1,100.000-1.000.000,131409.000000,-25183.65625
3718,2,PAYMENT,5.621400e+02,C837981622,7.221367e+05,7.215745e+05,M2053668237,0.000000e+00,0.000000e+00,no_fraud,1,0-1.000,-562.187500,0.00000
2919,2,CASH_IN,8.851071e+04,C2145921383,5.775554e+06,5.864064e+06,C977993101,1.180073e+06,7.442649e+05,no_fraud,1,10.000-100.000,88510.500000,-435807.81250
3957,3,CASH_IN,1.684791e+05,C1644155208,6.673927e+06,6.842406e+06,C575335780,3.086425e+05,1.401634e+05,no_fraud,1,100.000-1.000.000,168479.000000,-168479.09375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362584,741,TRANSFER,5.674548e+06,C992223106,5.674548e+06,5.674548e+06,C1366804249,0.000000e+00,0.000000e+00,fraud,31,1.000.000-10.000.000,0.000000,0.00000
6362608,742,TRANSFER,2.583554e+05,C1226129332,2.583554e+05,0.000000e+00,C1744173808,0.000000e+00,0.000000e+00,fraud,31,100.000-1.000.000,-258355.421875,0.00000
6362601,742,CASH_OUT,6.529939e+05,C1614818636,6.529939e+05,0.000000e+00,C362803701,0.000000e+00,6.529939e+05,fraud,31,100.000-1.000.000,-652993.937500,652993.93750


In [5]:
categorical_features = ['type', 'nameDest']
numeric_features = ['step', 'amount', 'oldbalanceOrig', 'oldbalanceDest', 'diffbalanceOrig', 'diffbalanceDest']
    

In [6]:
data = data.rename(columns={'oldbalanceOrg': 'oldbalanceOrig'})
data['diffbalanceOrig'] = data['newbalanceOrig'] - data['oldbalanceOrig']
data['diffbalanceDest'] = data['newbalanceDest'] - data['oldbalanceDest']
data = data.drop(columns=['newbalanceOrig', 'nameOrig', 'newbalanceDest'])
data['isFraud'] = data['isFraud'].map({'fraud': 1, 'no_fraud': 0})
data['type'] = data['type'].map({'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5})

In [7]:
X = data.drop(columns= 'isFraud')
y = data['isFraud']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [8]:
# categorical_transformer = OneHotEncoder(handle_unknown='ignore')
# numeric_transformer = StandardScaler()

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)
#     ]
# )


In [9]:
# X_train = preprocessor.fit_transform(X_train)
# X_train

In [10]:
# smote = SMOTE(random_state=42)
# X_train, y_train = smote.fit_resample(X_train, y_train)


In [11]:
# X_test = preprocessor.transform(X_test)
# X_test

In [12]:

with mlflow.start_run():
    mlflow.set_tag('developer', 'Maldu')

    # Crear el modelo base
    model = LogisticRegression(max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    # Evaluar el modelo en el conjunto de entrenamiento
    train_accuracy = model.score(X_train, y_train)
    mlflow.log_metric('train_accuracy', train_accuracy)
    print(f"Training Accuracy: {train_accuracy:.4f}")

    # Evaluar el modelo en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric('accuracy', accuracy)
    print(f"Test Accuracy: {accuracy:.4f}")

    test_accuracy = model.score(X_test, y_test)
    mlflow.log_metric('test_accuracy', test_accuracy)
    print(f"Test Accuracy (using score method): {test_accuracy:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['no_fraud', 'fraud'], yticklabels=['no_fraud', 'fraud'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig("confusion_matrix.png")  
    mlflow.log_artifact("confusion_matrix.png")  
    plt.close()
    print("Confusion Matrix saved and logged.")

    # Classification Report
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Log precision and recall for each class and average
    for key, value in report.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                if sub_key in ['precision', 'recall']:
                    mlflow.log_param(f'{key}_{sub_key}', sub_value)
                    print(f"{key} - {sub_key}: {sub_value:.4f}")
        else:
            if key in ['precision', 'recall']:
                mlflow.log_param(f'{key}', value)
                print(f"{key}: {value:.4f}")

    # Calculate ROC AUC
    y_prob = model.predict_proba(X_test)[:, 1]  
    roc_auc = roc_auc_score(y_test, y_prob)
    mlflow.log_metric('roc_auc', roc_auc)
    print(f"ROC AUC: {roc_auc:.4f}")

    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.savefig("roc_curve.png")  
    mlflow.log_artifact("roc_curve.png")  
    plt.close()
    print("ROC Curve saved and logged.")

    # Guardar el modelo
    mlflow.sklearn.log_model(model, "model")
    print(f"Model saved. Default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/08/18 12:45:56 INFO mlflow.tracking._tracking_service.client: 🏃 View run adventurous-pug-294 at: http://127.0.0.1:5000/#/experiments/1/runs/ea70e153a448460f9914cbe1627d9d57.
2024/08/18 12:45:56 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


ValueError: could not convert string to float: 'M1987883187'